# GRAPE for systems with Dissipation

In [1]:
# ruff: noqa

In [2]:
import jax.numpy as jnp
from feedback_grape.grape import *
from feedback_grape.utils.gates import *
from feedback_grape.utils.operators import *
from feedback_grape.utils.states import *
from feedback_grape.utils.superoperator import *
from feedback_grape.utils.tensor import *

In [3]:
Sx = sigmax()
Sy = sigmay()
Sz = sigmaz()
Sm = sigmam()
Si = identity(2)
had_gate = hadamard()
# Hamiltonian
Del = 0.1  # Tunnelling term
wq = 1.0  # Energy of the 2-level system.
H0 = 0.5 * wq * sigmaz() + 0.5 * Del * sigmax()

# Amplitude damping#
# Damping rate:
gamma = 5
L0 = liouvillian(H0, [jnp.sqrt(gamma) * Sm])

# sigma X control
LC_x = liouvillian(Sx)
# sigma Y control
LC_y = liouvillian(Sy)
# sigma Z control
LC_z = liouvillian(Sz)

# Drift
drift = L0
# Controls - different combinations can be tried
ctrls = [LC_z, LC_x]
# Number of ctrls
n_ctrls = len(ctrls)

# start point for the map evolution
E0 = sprepost(Si, Si)

# target for map evolution
E_targ = sprepost(had_gate, had_gate)

In [4]:
# Number of time slots
n_ts = 10
# Time allowed for the evolution
evo_time = 2

In [5]:
# result = optimize_pulse(
#     drift,
#     ctrls,
#     E0,
#     E_targ,
#     n_ts,
#     evo_time,
#     type="liouvillian",
#     optimizer="adam",
#     convergence_threshold=1e-16,
#     max_iter=10000,
#     learning_rate=0.1,
# )

In [6]:
result.final_fidelity

_tuplegetter(1, 'Alias for field number 1')

In [7]:
from feedback_grape.utils.solver import mesolve_1
from feedback_grape.utils.states import coherent

jump_ops = [jnp.sqrt(gamma) * Sm, jnp.sqrt(gamma * 2) * Sm.T]

H0 = 0.5 * wq * sigmaz() + 0.5 * Del * sigmax()
H_ctrl = [Sz, Sx]

tsave = jnp.linspace(0, evo_time, n_ts)

psi = coherent(2, alpha=1.0)
rho0 = psi @ psi.T.conj()

print(rho0.shape)

mesolve_result_2 = mesolve_1([H0] + H_ctrl, jump_ops, rho0, tsave)

(2, 2)


In [8]:
print(len([H0] + H_ctrl))

3


In [9]:
print(rho0)

[[0.36787944+0.j 0.36787944+0.j]
 [0.36787944+0.j 0.36787944+0.j]]


In [10]:
print(mesolve_result_2)

[[4.86295571e-01+0.j         2.08981579e-07+0.03157762j]
 [2.08981579e-07-0.03157762j 2.49463311e-01+0.j        ]]


In [11]:
print(tsave)

[0.         0.22222222 0.44444444 0.66666667 0.88888889 1.11111111
 1.33333333 1.55555556 1.77777778 2.        ]


In [12]:
for item in zip([H0] + H_ctrl, tsave):
    print(item)

(Array([[ 0.5 +0.j,  0.05+0.j],
       [ 0.05+0.j, -0.5 +0.j]], dtype=complex128), Array(0., dtype=float64))
(Array([[ 1.+0.j,  0.+0.j],
       [ 0.+0.j, -1.+0.j]], dtype=complex128), Array(0.22222222, dtype=float64))
(Array([[0.+0.j, 1.+0.j],
       [1.+0.j, 0.+0.j]], dtype=complex128), Array(0.44444444, dtype=float64))


In [ ]:
c_ops = (
    [  # c_ops for each decay index
        [
            tensor(identity(30), jnp.sqrt(0.01) * sigmam()),
        ],
        ["hi"],
    ],
)

In [16]:
print(c_ops[0][0])

[Array([[0. +0.j, 0. +0.j, 0. +0.j, ..., 0. +0.j, 0. +0.j, 0. +0.j],
       [0.1+0.j, 0. +0.j, 0. +0.j, ..., 0. +0.j, 0. +0.j, 0. +0.j],
       [0. +0.j, 0. +0.j, 0. +0.j, ..., 0. +0.j, 0. +0.j, 0. +0.j],
       ...,
       [0. +0.j, 0. +0.j, 0. +0.j, ..., 0. +0.j, 0. +0.j, 0. +0.j],
       [0. +0.j, 0. +0.j, 0. +0.j, ..., 0. +0.j, 0. +0.j, 0. +0.j],
       [0. +0.j, 0. +0.j, 0. +0.j, ..., 0. +0.j, 0.1+0.j, 0. +0.j]],      dtype=complex128)]


In [ ]:
c_ops_iter = iter(c_ops)
for _ in range(2):
    print(next(c_ops_iter))

[[Array([[0. +0.j, 0. +0.j, 0. +0.j, ..., 0. +0.j, 0. +0.j, 0. +0.j],
       [0.1+0.j, 0. +0.j, 0. +0.j, ..., 0. +0.j, 0. +0.j, 0. +0.j],
       [0. +0.j, 0. +0.j, 0. +0.j, ..., 0. +0.j, 0. +0.j, 0. +0.j],
       ...,
       [0. +0.j, 0. +0.j, 0. +0.j, ..., 0. +0.j, 0. +0.j, 0. +0.j],
       [0. +0.j, 0. +0.j, 0. +0.j, ..., 0. +0.j, 0. +0.j, 0. +0.j],
       [0. +0.j, 0. +0.j, 0. +0.j, ..., 0. +0.j, 0.1+0.j, 0. +0.j]],      dtype=complex128)], ['hi']]


StopIteration: 